In [39]:
from xml.etree.ElementTree import iterparse

In [1]:
from lxml import etree
import pymysql
import time
import re
import pandas as pd

In [2]:
def fast_iter(context, func, func1, *args, **kwargs):

    for event, elem in context:
        
        func(elem, func1, *args, **kwargs)
        # reset elements，clear the internal data of the element
        elem.clear()

    del context
    

In [230]:
def process_element_protocol(elem,func1):

    pro_id_list = []
    pro_type_list =[]
    pro_host_list =[]
    standard_protocoltype_list = ["expression","purification","crystallization"]
    standard_host_list = ["e. coli","e.coli","bl21","b834","de3","s. cerevisiae","s.cerevisiae","pichia pastoris","yeast",
                          "insect","sf9","hi5","hi-5","cellfree","cell-free","293-f","293f","m. maripaludis","m.maripaludis"]

    for protocols in elem.xpath('.//protocolList/protocol'):
        for pro in protocols.xpath('.//protocolId'):
            proid = pro.text
            for pro_type in protocols.xpath('.//protocolType'):
                #for spte in standard_protocoltype_list:
                if pro_type.text in standard_protocoltype_list:
                    pro_id_list.append(proid)
                    pro_type_list.append(pro_type.text)
                    
                    for pro_dets in protocols.xpath('.//protocolText'):
                        for ind in range(len(standard_host_list)):
                            if standard_host_list[ind] not in pro_dets.text.lower() and ind == len(standard_host_list) - 1:
                                pro_host_list.append('')
                            elif standard_host_list[ind] in pro_dets.text.lower():
                                pro_host_list.append(standard_host_list[ind])
                                break
                            else:
                                continue

                #else:
                    #break
    protocol_info_df = pd.DataFrame(columns=['protocol_id', 'protocol_type','host'])
    protocol_info_df['protocol_id'] = pro_id_list
    protocol_info_df['protocol_type'] = pro_type_list
    protocol_info_df['host'] = pro_host_list
    outputpath='protocol_info.csv'
    protocol_info_df.to_csv(outputpath,sep=',',index=True,header=True)
    return protocol_info_df


In [3]:
def compare(list1,list2):
    if len(list1) < len(list2):
        list1.append('NA')
    return list1

In [40]:
def process_element_protocol(elem,func1):

    pro_id_list = []
    pro_type_list =[]
    pro_host_list =[]
    standard_host_list = ["e. coli","e.coli","bl21","b834","de3","s. cerevisiae","s.cerevisiae","pichia pastoris","yeast",
                          "insect","sf9","hi5","hi-5","cellfree","cell-free","293-f","293f","m. maripaludis","m.maripaludis"]

    for protocols in elem.xpath('.//protocol'):
        for pro in protocols.xpath('.//protocolId'):
            proid = pro.text
            pro_id_list.append(proid)
            for pro_type in protocols.xpath('.//protocolType'):
                pro_type_list.append(pro_type.text)
                    
                for pro_dets in protocols.xpath('.//protocolText'):
                    for ind in range(len(standard_host_list)):
                        if standard_host_list[ind] not in pro_dets.text.lower() and ind == len(standard_host_list) - 1:
                            pro_host_list.append('NA')
                        elif standard_host_list[ind] in pro_dets.text.lower():
                            pro_host_list.append(standard_host_list[ind])
                            break
                        else:
                            continue   

                #else:
                    #break
    protocol_info_df = pd.DataFrame(columns=['protocol_id', 'protocol_type','host'])
    protocol_info_df['protocol_id'] = pro_id_list
    protocol_info_df['protocol_type'] = pro_type_list
    protocol_info_df['host'] = pro_host_list
    outputpath='protocol_info.csv'
    protocol_info_df.to_csv(outputpath,sep=',',index=False,header=True)
    return protocol_info_df

In [14]:
def process_element_trail(elem, func1):
    
    id_list = []
    protein_name_list = []
    lab_list = []
    target_status_list = []
    source_organism_list = []
    database_list = []
    trail_status_list = []
    trail_stopdes_list = []
    trail_his_status_list = []
    trail_protocol_list = []
    protocol_expresslevel_list = []
    trail_expresslevel_list = []
    
    for targets in elem.xpath('.//target'):
        # extract trail id
        database_dict = {}
        database_name_list = []
        database_id_list = []
        taid = str(targets.attrib).split(' ')[1][:-1].strip("'")
        targetnames = targets.find('targetName')
        if targetnames is None:
            targetname = 'NA'
        else:
            #for targetnames in targets.xpath('.//targetName'):
            targetname = targetnames.text
            
        for labs in targets.xpath('.//laboratoryList/lab'):
            lab = labs.text
        for target_stas in targets.xpath('.//status'):
            target_sta = target_stas.text
        for source_organisms in targets.xpath('.//targetSequenceList/targetSequence/sourceOrganism/scientificName'):
            source_organism = source_organisms.text
        for database_names in targets.xpath('.//databaseRefList/databaseRef/databaseName'):
            database_name_list.append(database_names.text)
        for database_ids in targets.xpath('.//databaseRefList/databaseRef/databaseId'):
            database_id_list.append(database_ids.text)
            
        database_dict = dict(zip(database_name_list,database_id_list))
        
        for trails in targets.xpath('.//trialList/trial'):
            trid = str(trails.attrib).split(' ')[1][:-1].strip("'")
            tid = taid+'-'+trid
            his_dic = {}
            pro_dic = {}
            for trail_stas in trails.xpath('.//status'):
                trail_sta = trail_stas.text
            
            for trail_his_ids in trails.xpath('.//statusHistoryList/statusHistory'):
                trail_his_id = str(trail_his_ids.attrib).split(' ')[1][:-1].strip("'")

                for trail_his_stas in trail_his_ids.xpath('.//status'):
                    his_dic[trail_his_id] = trail_his_stas.text
            if trail_sta == 'work stopped':
                for trail_sds in trails.xpath('.//stopDetails/stopStatus'):
                    trail_sd = trail_sds.text
                    trail_stopdes_list.append(trail_sd)
            
            id_list.append(tid)
            protein_name_list.append(targetname)
            lab_list.append(lab)
            target_status_list.append(target_sta)
            source_organism_list.append(source_organism)
            database_list.append(database_dict)
            trail_status_list.append(trail_sta)
            trail_his_status_list.append(his_dic)
            
            tra_pro_id_list = []
            tra_pro_type_list = []
            for tra_protocols in trails.xpath('.//trialProtocolList/protocolRef/protocolId'):
                tra_pro_id_list.append(tra_protocols.text)
            for tra_pro_types in trails.xpath('.//trialProtocolList/protocolRef/protocolType'):
                tra_pro_type_list.append(tra_pro_types.text)
            for pro_dets in trails.xpath('.//trialProtocolList/protocolRef/protocolDetails'):
                #pro_dets = tra_pro_lists.find('protocolDetails')
                if pro_dets is not None:
                    pro_exp_lev = re.findall(r'Cell Growth Protocol, Expression: (\w)',pro_dets.text, re.M)
                    if pro_exp_lev != []:
                        protocol_expresslevel_list.append(pro_exp_lev[0])

                        
            protocol_dict = dict(zip(tra_pro_type_list,tra_pro_id_list)) 
            trail_protocol_list.append(protocol_dict)

            
            for trail_oces in trails.xpath('.//trialOutcomeList/trialOutcome'):
                if '0' in str(trail_oces.attrib):
                    for trail_exp_lev in trail_oces.xpath('.//outcomeDetails'):
                        trail_exp_level = re.findall(r'Expression level  : (\w+)',trail_exp_lev.text, re.M)
                        if trail_exp_level != []:
                            trail_expresslevel_list.append(trail_exp_level[0])
                        
            #protein_name_list = func1(protein_name_lis,id_list)
            lab_list = func1(lab_list,id_list)
            source_organism_list = func1(source_organism_list,id_list)
            database_list = func1(database_list,id_list)
            trail_status_list = func1(trail_status_list,id_list)
            trail_his_status_list = func1(trail_his_status_list,id_list)
            trail_protocol_list = func1(trail_protocol_list,id_list)
            protocol_expresslevel_list = func1(protocol_expresslevel_list,id_list)
            trail_expresslevel_list = func1(trail_expresslevel_list,id_list)
            trail_stopdes_list = func1(trail_stopdes_list,id_list)

    
    trail_info_df = pd.DataFrame(columns=['trail_id', 'protein_name', 'lab', 'target_status', 'source_organism','database_info','trail_status','trail_stopdes','trail_his_status','trail_protocol_info','protocol_expresslevel','trail_expresslevel'])
    trail_info_df['trail_id'] = id_list
    trail_info_df['protein_name'] = protein_name_list
    trail_info_df['lab'] = lab_list
    trail_info_df['target_status'] = target_status_list
    trail_info_df['source_organism'] = source_organism_list
    trail_info_df['database_info'] = database_list
    trail_info_df['trail_status'] = trail_status_list
    trail_info_df['trail_stopdes'] = trail_stopdes_list
    trail_info_df['trail_his_status'] = trail_his_status_list
    trail_info_df['trail_protocol_info'] = trail_protocol_list
    trail_info_df['protocol_expresslevel'] = protocol_expresslevel_list
    trail_info_df['trail_expresslevel'] = trail_expresslevel_list
    outputpath='trail_info.csv'
    trail_info_df.to_csv(outputpath,sep=',',index=False,header=True)
    

In [10]:
def multi_proc(name):
    eis = etree.iterparse(name,events=('end',),encoding='UTF-8',tag='targetTrack')
    return eis

In [16]:
def multi_proc(name):
    eis = etree.iterparse(name,events=('end',),encoding='UTF-8',tag='targetTrack')
    return eis

if __name__ == '__main__':
    process_list = []
    for i in range(20):  #开启5个子进程执行fun1函数
        p = Process(target=multi_proc,args=('tt.xml',)) #实例化进程对象
        p.start()
        process_list.append(p)

    for i in process_list:
        p.join()

    print('结束测试')

结束测试


In [15]:
if __name__ == '__main__':
    print('start', time.strftime('%Y-%m-%d %H:%M:%S', time.localtime()))
    start = time.time()


    # By iteratively reading xml, the namespace with the namespace should be added
    context_protocol = etree.iterparse('tt.xml',events=('end',),encoding='UTF-8',tag='protocolList')
    context_trail = etree.iterparse('tt.xml',events=('end',),encoding='UTF-8',tag='targetTrack')
    # Quickly read xml data
    fast_iter(context_trail, process_element_trail, compare)
    fast_iter(context_protocol, process_element_protocol, compare)

    print('stop', time.strftime('%Y-%m-%d %H:%M:%S', time.localtime()))
    print('time', time.time()-start)

start 2022-01-17 18:25:04
stop 2022-01-17 18:44:47
time 1182.3576431274414
